<a href="https://colab.research.google.com/github/N1a2t3a/.Data_Science/blob/main/HW5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report
import zipfile
import io
import os
import glob

In [33]:
zip_file_path = '/content/homework.zip'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall('/content/homework')

In [34]:
common_length = 240
data = {
    'timestamp': pd.date_range(start='2022-01-01', periods=common_length, freq='1H'),
    'accelerometer_X': np.random.randn(common_length),
    'accelerometer_Y': np.random.randn(common_length),
    'accelerometer_Z': np.random.randn(common_length),
    'activity_label': np.random.choice(['idle', 'running', 'stairs', 'walking'], size=common_length)
}

df = pd.DataFrame(data)

print(df.head())

            timestamp  accelerometer_X  accelerometer_Y  accelerometer_Z  \
0 2022-01-01 00:00:00         1.147132        -0.316513        -0.315541   
1 2022-01-01 01:00:00        -0.476852         0.576465        -0.455895   
2 2022-01-01 02:00:00         0.456723        -0.064500        -0.008639   
3 2022-01-01 03:00:00        -0.446095         0.835776         0.621824   
4 2022-01-01 04:00:00         0.022456         1.743881         0.142865   

  activity_label  
0        walking  
1        walking  
2        running  
3        walking  
4        running  


In [35]:
if 'timestamp' in df.columns:
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df.set_index('timestamp', inplace=True)

    time_features = df.resample('1T').agg(['mean', 'std', 'max', 'min'])

    print("Processed Dataset Sample:\n", time_features.head())
else:
    print("Column 'timestamp' not found in the DataFrame.")

Processed Dataset Sample:
                     accelerometer_X                         accelerometer_Y  \
                               mean std       max       min            mean   
timestamp                                                                     
2022-01-01 00:00:00        1.147132 NaN  1.147132  1.147132       -0.316513   
2022-01-01 00:01:00             NaN NaN       NaN       NaN             NaN   
2022-01-01 00:02:00             NaN NaN       NaN       NaN             NaN   
2022-01-01 00:03:00             NaN NaN       NaN       NaN             NaN   
2022-01-01 00:04:00             NaN NaN       NaN       NaN             NaN   

                                            accelerometer_Z                \
                    std       max       min            mean std       max   
timestamp                                                                   
2022-01-01 00:00:00 NaN -0.316513 -0.316513       -0.315541 NaN -0.315541   
2022-01-01 00:01:00 NaN       Na

<ipython-input-35-7d997f432106>:5: FutureWarning: ['activity_label'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  time_features = df.resample('1T').agg(['mean', 'std', 'max', 'min'])


In [37]:
fixed_feature_count = 100

data_folder = 'homework/data/'

features = []
labels = []

actions = ['idle', 'running', 'stairs', 'walking']

for action in actions:
    action_folder = os.path.join(data_folder, action)

    for file_path in glob.glob(os.path.join(action_folder, '*.csv')):
        df = pd.read_csv(file_path).iloc[:, :fixed_feature_count]
        features.append(df)

        labels.append(action)

data = pd.concat(features, axis=0, ignore_index=True)

data = data.iloc[:len(labels)]

labels_df = pd.DataFrame(labels, columns=['label'])

X_train, X_test, y_train, y_test = train_test_split(data, labels_df, test_size=0.2, random_state=42)


In [38]:
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)
X_train_scaled, X_test_scaled = X_train, X_test

In [39]:
svm_model = SVC()
svm_model.fit(X_train_scaled, y_train)

rf_model = RandomForestClassifier()
rf_model.fit(X_train_scaled, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
<ipython-input-39-ae2f747709b4>:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_model.fit(X_train_scaled, y_train)


RandomForestClassifier()

In [40]:
svm_predictions = svm_model.predict(X_test_scaled)
print("SVM Classification Report:\n", classification_report(y_test, svm_predictions))

rf_predictions = rf_model.predict(X_test_scaled)
print("Random Forest Classification Report:\n", classification_report(y_test, rf_predictions))

SVM Classification Report:
               precision    recall  f1-score   support

        idle       0.00      0.00      0.00       220
     running       0.53      1.00      0.69       689
      stairs       0.00      0.00      0.00        30
     walking       0.00      0.00      0.00       354

    accuracy                           0.53      1293
   macro avg       0.13      0.25      0.17      1293
weighted avg       0.28      0.53      0.37      1293

Random Forest Classification Report:
               precision    recall  f1-score   support

        idle       0.13      0.09      0.11       220
     running       0.54      0.64      0.59       689
      stairs       0.00      0.00      0.00        30
     walking       0.24      0.21      0.23       354

    accuracy                           0.41      1293
   macro avg       0.23      0.23      0.23      1293
weighted avg       0.38      0.41      0.39      1293



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
